2번문제 : 
고객 3,500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 
후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 
다음과 같은 형식의 CSV 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 
채점)

제출형식
```bash
custid,gender
 3500,0.267
 3501,0.578
 3502,0.88
```

In [34]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

path = "type_2/"
x_train_df = pd.read_csv(path+"X_train.csv",encoding='euc-kr')
x_test_df = pd.read_csv(path+"X_test.csv",encoding='euc-kr')
y_train_df = pd.read_csv(path+"y_train.csv",encoding='euc-kr')

## 1. 데이터분석

# 주구매상품과 주구매지점은 실수값이 아니라, 상관관계분석 불가하기에 변환필요
purchase_encoder = LabelEncoder()
train_purchase = x_train_df["주구매상품"]
test_purchase = x_test_df["주구매상품"]
purchase_encoder.fit(train_purchase)
train_purchase = purchase_encoder.transform(train_purchase)
test_purchase = purchase_encoder.transform(test_purchase)

position_encoder = LabelEncoder()
train_position = x_train_df["주구매지점"]
test_position = x_test_df["주구매지점"]
position_encoder.fit(train_position)
train_position = position_encoder.transform(train_position)
test_position = position_encoder.transform(test_position)

x_train_df["주구매상품"] = train_purchase
x_train_df["주구매지점"] = train_position
x_test_df["주구매상품"] = test_purchase
x_test_df["주구매지점"] = test_position

all_df = pd.concat([x_train_df,y_train_df.iloc[:,1:]],axis=1)
# all_df.corr()["gender"]


## 2. 데이터전처리
x_train_df.drop(columns=['환불금액','주구매상품','주구매지점','내점당구매건수','주말방문비율','구매주기'],axis=1,inplace=True)
x_test_df.drop(columns=['환불금액','주구매상품','주구매지점','내점당구매건수','주말방문비율','구매주기'],axis=1,inplace=True)

x_train_data = x_train_df.iloc[:,1:]
x_test_data = x_test_df.iloc[:,1:]
y_train_data = y_train_df.iloc[:,1:].values.reshape(-1)

scaler = MinMaxScaler()
scaler.fit(x_train_data)
x_train_data = scaler.transform(x_train_data)
x_test_data = scaler.transform(x_test_data)

## 3. test model
x_train_temp,x_test_temp,y_train_temp,y_test_temp = train_test_split(x_train_data,
                                                                     y_train_data,
                                                                     train_size=0.8,
                                                                     random_state=2)
# 로지스틱회귀를 통한 예측성능평가
test_model = LogisticRegression()
test_model.fit(x_train_temp,y_train_temp)
prediction = test_model.predict_proba(x_test_temp)
result = prediction[:,1]
print(roc_auc_score(y_test_temp,result))

# 랜덤포레스트를 통한 예측성능평가
test_model = RandomForestClassifier()
test_model.fit(x_train_temp,y_train_temp)
prediction = test_model.predict_proba(x_test_temp)
result = prediction[:,1]
print(roc_auc_score(y_test_temp,result))

## 4. submission model
final_model = LogisticRegression()
final_model.fit(x_train_data,y_train_data)
prediction = final_model.predict_proba(x_test_data)
result = prediction[:,1]

cust_id_lst = x_test_df["cust_id"]
output_df = pd.DataFrame({"cust_id":cust_id_lst,'gender':result})
output_df.set_index("cust_id",inplace=True)
output_df.to_csv('수험번호.csv',index=False)

0.6497254656658327
0.5728819154851265
